In [1]:
from pinecone import Pinecone, PodSpec 
from langchain.document_loaders import PyMuPDFLoader


from IPython.display import Markdown, display

f:\Jupyter\Upwork\DocumentQA\DocumentQA\venv\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:

from hybrid_pinecone import HybridPinecone

### Load Data
We use langchain PDF loaders to read pdfs. 

In [10]:
filepath = "data/E1. ExngTextOnly.pdf"
loader = PyMuPDFLoader(filepath)
documents = loader.load()

In [11]:
contexts = [document.page_content for document in documents]

### Sparse and Dense Embeddings

In [8]:
from tqdm.auto import tqdm

from transformers import BertTokenizerFast              # Sparse Embeddings
from sentence_transformers import SentenceTransformer   # Dense Embeddings
from collections import Counter

In [12]:
# SPARSE EMBEDDINGS

# load bert tokenizer from huggingface

tokenizer = BertTokenizerFast.from_pretrained(
    'bert-base-uncased'
)

inputs = tokenizer(
    contexts[0], padding=True, truncation=True,
    max_length=512
)
inputs.keys()


# extract the input ids
input_ids = inputs['input_ids']

# convert the input_ids list to a dictionary of key to frequency values
sparse_vec = dict(Counter(input_ids))
sparse_vec
def build_dict(input_batch):
 # store a batch of sparse embeddings
   sparse_emb = []
   # iterate through input batch
   for token_ids in input_batch:
       indices = []
       values = []
       # convert the input_ids list to a dictionary of key to frequency values
       d = dict(Counter(token_ids))
       for idx in d:
            indices.append(idx)
            values.append(float(d[idx]))                        # Extremely important to cast values as float
                                                                # Otherwise you get: SparseValuesMissingKeysError: Missing required keys in data in column `sparse_values`.
       sparse_emb.append({'indices': indices, 'values': values})
   # return sparse_emb list
   return sparse_emb

def generate_sparse_vectors(context_batch):
    """
    create batch of input_ids required by pinecone for sparse vector sotrage
    """
    inputs = tokenizer(
            context_batch, padding=True,
            truncation=True,
            max_length=512
    )['input_ids']
    # create sparse dictionaries
    sparse_embeds = build_dict(inputs)
    return sparse_embeds

In [15]:
# DENSE EMBEDDINGS

# load a sentence transformer model from huggingface
model = SentenceTransformer(
    'multi-qa-MiniLM-L6-cos-v1',
    device='cpu'                    # or cuda, if available
)
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

### Initialize Vector Database

In [24]:
pc = HybridPinecone(
    api_key = "3c9fd500-de4d-40ed-8c77-12302b71e8ce",  # app.pinecone.io
    environment = "gcp-starter"
)

In [3]:
index_name = "hybrid-test"

In [27]:


# create the index
pc.create_index(
   index_name = index_name,
   dimension = 384,  # dimensionality of dense model
   pod_type='p1',
   metric = "dotproduct",
)

<Response [201]>

In [4]:
pc = Pinecone(
    api_key = "3c9fd500-de4d-40ed-8c77-12302b71e8ce",  # app.pinecone.io
    environment = "gcp-starter"
)
index = pc.Index(name=index_name)

In [32]:
batch_size = 50

for i in tqdm(range(0, len(contexts), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(contexts))
    # extract batch
    context_batch = contexts[i:i_end]
    # create unique IDs
    ids = [str(x) for x in range(i, i_end)]
    # add context passages as metadata
    meta = [{'context': context} for context in context_batch]
    # create dense vectors
    dense_embeds = model.encode(context_batch).tolist()
    # create sparse vectors
    sparse_embeds = generate_sparse_vectors(context_batch)

    vectors = []
    # loop through the data and create dictionaries for uploading documents to pinecone index
    for _id, sparse, dense, metadata in zip(ids, sparse_embeds, dense_embeds, meta):
        vectors.append({
            'id': _id,
            'sparse_values': sparse,
            'values': dense,
            'metadata': metadata
        })

    # upload the documents to the new hybrid index
    index.upsert(vectors)

# show index description after uploading the documents
index.describe_index_stats()

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:03<00:00,  3.61s/it]


{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [37]:
# def hybrid_query(question, top_k, alpha):
#     # convert the question into a sparse vector
#     sparse_vec = generate_sparse_vectors([question])
#     # convert the question into a dense vector
#     dense_vec = model.encode([question]).tolist()
#     # set the query parameters to send to pinecone
#     query = {
#       "topK": top_k,
#       "vector": dense_vec,
#       "sparseVector": sparse_vec[0],
#       "alpha": alpha,
#       "includeMetadata": True
#     }
#     # query pinecone with the query parameters
#     result = index.query(query)
#     # return search results as json
#     return result

In [29]:
question = "What did willy wonka do with vitawonk?"

In [28]:
sparse = generate_sparse_vectors([question])[0]
alpha = 0.4
[v * (1 - alpha) for v in sparse['values']]

[0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6]

In [36]:
dense = model.encode([question]).tolist()[0]
[v * alpha for v in dense]

[-0.022238956391811372,
 0.062448978424072266,
 0.006351868063211441,
 -0.0016900459304451942,
 -0.02263036072254181,
 0.02233632951974869,
 0.019580307602882388,
 0.007697943598031998,
 -0.007668163627386093,
 0.00793217122554779,
 -0.011692066490650178,
 -0.0307610422372818,
 0.002302438206970692,
 -0.021654368937015535,
 0.023146696388721466,
 -0.004521287232637406,
 0.00912071466445923,
 0.018840651214122775,
 -0.016990162432193756,
 -0.026402932405471802,
 -0.0208988755941391,
 -0.00027084730099886657,
 0.016845071315765382,
 0.0035627253353595735,
 -0.01064179316163063,
 0.002639190107584,
 0.029694929718971252,
 0.0060277082026004795,
 0.006275960057973862,
 -0.02002657949924469,
 -0.024647326767444612,
 -0.004160820692777634,
 -0.014814341068267824,
 -0.0073359027504920965,
 -0.02858361899852753,
 0.020740592479705812,
 -0.009042345732450486,
 0.011702513694763184,
 -0.025087690353393557,
 -0.013969555497169495,
 0.028975486755371094,
 -0.011495745927095414,
 0.0028424883261322

In [37]:
def hybrid_scale(dense, sparse, alpha: float):
    # check alpha value is in range
    if alpha < 0 or alpha > 1:
        raise ValueError("Alpha must be between 0 and 1")
    # scale sparse and dense vectors to create hybrid search vecs
    hsparse = {
        'indices': sparse['indices'],
        'values':  [v * (1 - alpha) for v in sparse['values']]
    }
    hdense = [v * alpha for v in dense]
    return hdense, hsparse


def hybrid_query(question, top_k, alpha):
   # convert the question into a sparse vector
   sparse_vec = generate_sparse_vectors([question])[0]
   # convert the question into a dense vector
   dense_vec = model.encode([question]).tolist()[0]
   # scale alpha with hybrid_scale
   dense_vec, sparse_vec = hybrid_scale(
      dense_vec, sparse_vec, alpha
   )
   # query pinecone with the query parameters
   result = index.query(
      top_k=top_k,
      vector=dense_vec,
      sparse_vector=sparse_vec,
      include_metadata=True
   )
   # return search results as json
   return result

In [38]:
question = "What did willy wonka do with vitawonk?"

# %% [markdown]
# First, we will do a pure semantic search by setting the alpha value as 1.

# %%
hybrid_query(question, top_k=3, alpha=0.4)

{'matches': [{'id': '0',
              'metadata': {'context': 'Who are the oldest people you know? '
                                      'What are the\n'
                                      'oldest things you have (i) in your '
                                      'house, (ii) in your city,\n'
                                      'town or village? How old are they?\n'
                                      'Have you ever wished that you were '
                                      'older? Have\n'
                                      'you wished that you could grow up in a '
                                      'hurry?\n'
                                      'Mr Willy Wonka begins by inventing '
                                      'Wonka-\n'
                                      'Vite, which makes people younger. But '
                                      'Wonka-\n'
                                      'Vite is too strong. So some people '
                                  